In [30]:
import sys
import numpy as np

import cantera as ct
import struct
import os

In [50]:
# The simulation integrates in time for these values:
nstep = 10000
step_size = 1.e-7

# Defines the homogeneous mixture properties for use in the reactor
# COH2.cti is the chemical input file generated from the CHEMKIN files

#To generate the .cti file do: (see> man ck2cti)
#       ck2cti --input=chem.inp --thermo=therm.dat --transport=trans.dat 

gas = ct.Solution('chem_Lietal.cti')
# gas = ct.Solution('LiDryer_H2.cti')

In [51]:
data_1bar = np.zeros(shape=(nstep,11))

In [52]:
T_in = 1132
P_in = ct.one_atm * 5
gas.TP = T_in, P_in
phi=0.35
#g.X = 'H2:0.00811, O2:0.18316, N2:0.75691, H2O:0.05182'
#print(g.get_equivalence_ratio('O2'))
gas.set_equivalence_ratio(phi, {'H2':1.0}, {'O2':0.14440, 'N2':0.68155, 'H2O':0.07256})
# gas.set_equivalence_ratio(phi, {'H2':1.0}, {'O2':1, 'N2':3.76})


In [53]:
gas.viscosity

4.57152066502493e-05

In [54]:
gas.mean_molecular_weight

25.234744825378407

In [57]:
gas.cp/gas.cv

1.3221433143111903

In [39]:
# 'r' is the name of the object which repreesntes your reactor
# Here, r is a constant pressure reactor which solves the conservation 
# of species and energy equation
# The mixture properties are passed into the call to set the initial condition
r = ct.IdealGasConstPressureReactor(gas)

# sim is the object which represents the progress in time of the reactor
sim = ct.ReactorNet([r])
time = 0.0


In [40]:
#Here output is just to screen, can call routines to save to .csv or other files
# see numpy documentation

# To get ignition delay time, store array of T and time, define ignition as, say
# max(dT/dt)
dtdT = np.zeros(shape=(nstep,2))
grdT = np.zeros(shape=(nstep,1))

#print('%10s %10s %10s %14s' % ('t [s]','T [K]','P [Pa]','u [J/kg]'))
for n in range(nstep):
    time += step_size
# Integrates the solution in time
    sim.advance(time)
    data_1bar[n,0:9]=r.Y
    data_1bar[n,9]=r.T
#    print('%10.3e %10.3f %10.3f %14.6e' % (sim.time, r.T,
#                                           r.thermo.P, r.thermo.u))
#    f2.write('%10.3e %10.3f %10.3f %14.6e' % (sim.time, r.T,
#                                           r.thermo.P, r.thermo.u))
#    f2.write('\n')
#    f3.write('%16.9e %16.9e %16.9e %16.9e %16.9e %16.9e %16.9e %16.9e ' % (r.Y[0], r.Y[1], r.Y[2], r.Y[3], r.Y[4], r.Y[5], r.Y[6], r.Y[7]))
#    f3.write('\n')

    dtdT[n,0] = sim.time
    dtdT[n,1] = r.T
data_1bar[:,10] = P_in

In [41]:
max_index = np.argmax(np.gradient(dtdT[:,1]/np.gradient(dtdT[:,0])))
print(max_index)
print('autoignition delay time at Temp:%10.3f and pressure:%10.3f is %14.6e ms' % (T_in, P_in, dtdT[max_index,0]*1000))

1965
autoignition delay time at Temp:  1132.000 and pressure:506625.000 is   1.966000e-01 ms


In [45]:
ct.Transport

cantera._cantera.Transport

In [49]:
gas.viscosity

6.313824232094791e-05

In [9]:
data_1bar

array([[8.07857834e-03, 1.83176268e-01, 1.44438065e-12, ...,
        7.56923565e-01, 1.13200000e+03, 5.06625000e+05],
       [8.07857833e-03, 1.83176268e-01, 4.98539936e-12, ...,
        7.56923565e-01, 1.13200000e+03, 5.06625000e+05],
       [8.07857833e-03, 1.83176268e-01, 9.80480801e-12, ...,
        7.56923565e-01, 1.13200000e+03, 5.06625000e+05],
       ...,
       [1.55128555e-06, 1.18829958e-01, 1.50938998e-05, ...,
        7.56923565e-01, 1.84245769e+03, 5.06625000e+05],
       [1.55128555e-06, 1.18829958e-01, 1.50938998e-05, ...,
        7.56923565e-01, 1.84245769e+03, 5.06625000e+05],
       [1.55128555e-06, 1.18829958e-01, 1.50938998e-05, ...,
        7.56923565e-01, 1.84245769e+03, 5.06625000e+05]])

In [15]:
data_1bar.shape

(1000000, 11)

In [18]:
data_1bar_u = data_1bar.reshape(-1, order='F')

In [43]:
data_1bar

array([[8.07817135e-03, 1.83182523e-01, 2.79184652e-14, ...,
        7.56916438e-01, 1.10000000e+03, 1.01325000e+05],
       [8.07817135e-03, 1.83182523e-01, 1.08960151e-13, ...,
        7.56916438e-01, 1.10000000e+03, 1.01325000e+05],
       [8.07817135e-03, 1.83182523e-01, 2.40335991e-13, ...,
        7.56916438e-01, 1.10000000e+03, 1.01325000e+05],
       ...,
       [2.62205595e-06, 1.18775654e-01, 2.54109842e-05, ...,
        7.56916438e-01, 1.81158511e+03, 1.01325000e+05],
       [2.62205595e-06, 1.18775654e-01, 2.54109842e-05, ...,
        7.56916438e-01, 1.81158511e+03, 1.01325000e+05],
       [2.62205595e-06, 1.18775654e-01, 2.54109842e-05, ...,
        7.56916438e-01, 1.81158511e+03, 1.01325000e+05]])

In [35]:
f = 'first.mpi'
fout = open(f, 'ab')
temp=fout.write(struct.pack('<11000000d', *data_1bar_u))
# seek = temp+seek
fout.seek(temp)

88000000

In [37]:
fout.close()

In [41]:
f = 'first.mpi'
data = np.fromfile(f, dtype="float64",count=nstep*11)
data = np.reshape(data, newshape=(nstep, 11), order='F')

In [44]:
data-data_1bar

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])